  # This notebook splits the data into training, validation and test sets. Also pre-processes the data
  Data is split by instance, for each problem set 6 instances are used for training, 2 for validation, and 1 for testing

In [58]:
import pandas as pd

In [59]:
column_names = [
    'iterations',
    'instance_type',
    'tw_spread',
    'operator_selection_mechanism',
    'number_of_vertices_to_remove',
    "delta_last_improv", #change to rel_delta... for future work when i want to test on my benchmarks
    "acceptance_ratio",
    "i_last_improv",
    'prev_remove_operator',
    'prev_insert_operator',
    'route_imbalance',
    'capacity_utilization',
    'success_r_op_1',
    'success_r_op_2',
    'success_r_op_3',
    'success_r_op_4',
    'success_r_op_5',
    'success_i_op_1',
    'success_i_op_2',
    'success_i_op_3',
    'delta_cost',
    'new_cost',
    'chosen_remove_operator',
    'chosen_insert_operator'
]


log_df = pd.read_csv("../training_alns_iterations.log", sep=',', header=None, names=column_names)

In [60]:
log_df['iterations'] = log_df['iterations'].apply(lambda x: int(str(x).split(":")[-1]))

In [61]:
log_df['current_cost'] = log_df['new_cost'] - log_df['delta_cost']

In [62]:
#relativise delta last improv:
log_df['rel_delta_last_improv'] = log_df['delta_last_improv'] / log_df['current_cost']

In [7]:
log_df[['delta_last_improv', 'current_cost', 'rel_delta_last_improv']].head()

,delta_last_improv,current_cost,rel_delta_last_improv
0,0.000000,10267.032510,0.000000
1,-718.948672,9548.083839,-0.075298
2,-91.810910,9456.272928,-0.009709
3,-160.354919,9295.918009,-0.017250
4,-160.354919,9295.918009,-0.017250


In [64]:
def compute_short_long_term_improvement(df, alpha, k):
    improvements = []

    for i in range(len(df)):
        current_cost = df.loc[i, 'current_cost']
        immediate = df.loc[i, 'delta_cost']
        future_window = df['delta_cost'].iloc[i+1:i+1+k]

        # Best improvement (most negative value) in the next k steps
        best_future_improvement = future_window.min() if not future_window.empty else 0
        #todo: try to do the average instead of min
        #best_future_improvement = future_window.mean() if not future_window.empty else 0

        # Weighted combination
        rel_immediate = immediate / current_cost #added
        rel_best_future_improvement = best_future_improvement / current_cost #added
        combined = alpha * rel_immediate + (1 - alpha) * rel_best_future_improvement
        improvements.append(combined)

    df['short_long_improvement'] = improvements
    return df

In [65]:
df = compute_short_long_term_improvement(log_df, alpha=0.5, k=10)

In [9]:
# original short_long_term_improvement
df.head()

,iterations,instance_type,tw_spread,operator_selection_mechanism,number_of_vertices_to_remove,delta_last_improv,acceptance_ratio,i_last_improv,prev_remove_operator,prev_insert_operator,...,success_i_op_1,success_i_op_2,success_i_op_3,delta_cost,new_cost,chosen_remove_operator,chosen_insert_operator,current_cost,rel_delta_last_improv,short_long_improvement
0,1,0,1,1,15,0.000000,0.0,0,NaN,NaN,...,1.0,1.0,1.0,-718.948672,9548.083839,1,2,10267.032510,0.000000,-0.054251
1,2,0,1,1,5,-718.948672,1.0,0,1.0,2.0,...,1.0,1.0,1.0,-91.810910,9456.272928,1,3,9548.083839,-0.075298,-0.012359
2,3,0,1,1,5,-91.810910,1.0,0,1.0,3.0,...,1.0,1.0,1.0,-160.354919,9295.918009,4,1,9456.272928,-0.009709,-0.017553
3,4,0,1,1,5,-160.354919,1.0,0,4.0,1.0,...,1.0,1.0,1.0,0.000000,9295.918009,5,2,9295.918009,-0.017250,-0.005781
4,5,0,1,1,5,-160.354919,1.0,1,5.0,2.0,...,1.0,1.0,1.0,0.000000,9295.918009,3,1,9295.918009,-0.017250,-0.005781


In [66]:
# 2nd short_long_term_improvement with alpha=0.5, k=10
df.head()

,iterations,instance_type,tw_spread,operator_selection_mechanism,number_of_vertices_to_remove,delta_last_improv,acceptance_ratio,i_last_improv,prev_remove_operator,prev_insert_operator,...,success_i_op_1,success_i_op_2,success_i_op_3,delta_cost,new_cost,chosen_remove_operator,chosen_insert_operator,current_cost,rel_delta_last_improv,short_long_improvement
0,1,0,1,1,15,0.000000,0.0,0,NaN,NaN,...,1.0,1.0,1.0,-718.948672,9548.083839,1,2,10267.032510,0.000000,-0.043736
1,2,0,1,1,5,-718.948672,1.0,0,1.0,2.0,...,1.0,1.0,1.0,-91.810910,9456.272928,1,3,9548.083839,-0.075298,-0.014188
2,3,0,1,1,5,-91.810910,1.0,0,1.0,3.0,...,1.0,1.0,1.0,-160.354919,9295.918009,4,1,9456.272928,-0.009709,-0.017950
3,4,0,1,1,5,-160.354919,1.0,0,4.0,1.0,...,1.0,1.0,1.0,0.000000,9295.918009,5,2,9295.918009,-0.017250,-0.009635
4,5,0,1,1,5,-160.354919,1.0,1,5.0,2.0,...,1.0,1.0,1.0,0.000000,9295.918009,3,1,9295.918009,-0.017250,-0.009635


In [21]:
# 3rd short_long_term_improvement with alpha=0.5, k=10 and average instead of min
df.head()

,iterations,instance_type,tw_spread,operator_selection_mechanism,number_of_vertices_to_remove,delta_last_improv,acceptance_ratio,i_last_improv,prev_remove_operator,prev_insert_operator,...,success_i_op_1,success_i_op_2,success_i_op_3,delta_cost,new_cost,chosen_remove_operator,chosen_insert_operator,current_cost,rel_delta_last_improv,short_long_improvement
0,1,0,1,1,15,0.000000,0.0,0,NaN,NaN,...,1.0,1.0,1.0,-718.948672,9548.083839,1,2,10267.032510,0.000000,-0.036528
1,2,0,1,1,5,-718.948672,1.0,0,1.0,2.0,...,1.0,1.0,1.0,-91.810910,9456.272928,1,3,9548.083839,-0.075298,-0.005957
2,3,0,1,1,5,-91.810910,1.0,0,1.0,3.0,...,1.0,1.0,1.0,-160.354919,9295.918009,4,1,9456.272928,-0.009709,-0.008791
3,4,0,1,1,5,-160.354919,1.0,0,4.0,1.0,...,1.0,1.0,1.0,0.000000,9295.918009,5,2,9295.918009,-0.017250,-0.000318
4,5,0,1,1,5,-160.354919,1.0,1,5.0,2.0,...,1.0,1.0,1.0,0.000000,9295.918009,3,1,9295.918009,-0.017250,-0.000318


In [12]:
df.count()
# all columns are complete except for pre_remove_operator and prev_insert_operator since the first iteration in every alns run has no previous values

iterations                      216000
instance_type                   216000
tw_spread                       216000
operator_selection_mechanism    216000
number_of_vertices_to_remove    216000
delta_last_improv               216000
acceptance_ratio                216000
i_last_improv                   216000
prev_remove_operator            215892
prev_insert_operator            215892
route_imbalance                 216000
capacity_utilization            216000
success_r_op_1                  216000
success_r_op_2                  216000
success_r_op_3                  216000
success_r_op_4                  216000
success_r_op_5                  216000
success_i_op_1                  216000
success_i_op_2                  216000
success_i_op_3                  216000
delta_cost                      216000
new_cost                        216000
chosen_remove_operator          216000
chosen_insert_operator          216000
current_cost                    216000
rel_delta_last_improv    

In [68]:
# 2nd short_long_term_improvement
improv_per_instance_and_operator = df.groupby(['instance_type', 'tw_spread', 'chosen_remove_operator']).short_long_improvement.agg(['min','max', 'sum'])
improv_per_instance_and_operator

min       max         sum
instance_type tw_spread chosen_remove_operator                                
0             1         0                      -0.178599  0.058765 -354.105009
                        1                      -0.163052  0.064596 -281.619133
                        2                      -0.160416  0.094666 -343.775971
                        3                      -0.200709  0.043954 -510.262170
                        4                      -0.148368  0.133559  -86.947375
              2         0                      -0.176021  0.087061 -191.445985
                        1                      -0.159483  0.071227 -172.444642
                        2                      -0.159942  0.140243 -165.508027
                        3                      -0.182167  0.088369 -254.428240
                        4                      -0.162495  0.153432 -170.940414
1             1         0                      -0.094667  0.039508 -170.213152
                        1                      -0.087064  0.048561 -170.465362
                        2                      -0.084101  0.043242 -152.993865
                        3                      -0.112698  0.025468 -277.121080
                        4                      -0.088455  0.055404  -79.019085
              2         0                      -0.198590  0.110032 -264.563137
                        1                      -0.230306  0.132440 -273.712666
                        2                      -0.229738  0.152470 -243.050895
                        3                      -0.218311  0.098723 -321.159757
                        4                      -0.245717  0.174848 -269.512194
2             1         0                      -0.083643  0.023650 -141.706079
                        1                      -0.074223  0.038847 -133.241486
                        2                      -0.087995  0.039431 -141.713283
                        3                      -0.090302  0.020089 -263.019793
                        4                      -0.074458  0.052262  -97.905712
              2         0                      -0.230378  0.086536 -206.340345
                        1                      -0.250136  0.074294 -218.513944
                        2                      -0.233331  0.108434 -207.710176
                        3                      -0.255777  0.089493 -365.314484
                        4                      -0.260432  0.146007 -200.779614

In [22]:
# 3rd short_long_term_improvement
improv_per_instance_and_operator = df.groupby(['instance_type', 'tw_spread', 'chosen_remove_operator']).short_long_improvement.agg(['min','max', 'sum'])
improv_per_instance_and_operator

min       max         sum
instance_type tw_spread chosen_remove_operator                                
0             1         1                      -0.086876  0.113992  -15.550456
                        2                      -0.067392  0.107397   21.437002
                        3                      -0.087472  0.139678   -5.339235
                        4                      -0.120995  0.099797 -140.894170
                        5                      -0.068028  0.169876  178.169823
              2         1                      -0.095595  0.118735    6.377131
                        2                      -0.079921  0.098096    5.618225
                        3                      -0.095234  0.187691   15.512803
                        4                      -0.108633  0.114626  -36.931531
                        5                      -0.102717  0.224201   16.207973
1             1         1                      -0.063729  0.051801    5.767096
                        2                      -0.041821  0.072197    4.625273
                        3                      -0.059203  0.073602   17.211189
                        4                      -0.062456  0.046852  -78.107997
                        5                      -0.047648  0.087540   61.795552
              2         1                      -0.148583  0.193351   16.457497
                        2                      -0.102436  0.216278   -2.135381
                        3                      -0.104449  0.236551   29.492213
                        4                      -0.113427  0.169227  -33.590815
                        5                      -0.119162  0.233265   16.854713
2             1         1                      -0.042499  0.051162   11.037325
                        2                      -0.041186  0.068462   16.256848
                        3                      -0.045691  0.061256   16.447162
                        4                      -0.058419  0.036777  -77.188751
                        5                      -0.043172  0.063376   42.702875
              2         1                      -0.156126  0.185266   28.376039
                        2                      -0.130525  0.130343   15.593677
                        3                      -0.140135  0.209735   29.387931
                        4                      -0.120671  0.111616  -87.025718
                        5                      -0.112660  0.258543   35.963020

In [67]:
# correct operator indexes:
df['prev_remove_operator'] = df['prev_remove_operator']-1
df['prev_insert_operator'] = df['prev_insert_operator']-1
df['chosen_remove_operator'] = df['chosen_remove_operator']-1
df['chosen_insert_operator'] = df['chosen_insert_operator']-1

In [70]:
df.shape

(216000, 27)

## Splitting in train, validation and test sets

In [71]:
problem_types = 6
problem_instances = 9
runs_per_instance = 2
iterations_per_run = 2000

In [72]:
# 1. divide dataframe by problem types
training_set, validation_set, test_set = [], [], []
problem_size = problem_instances * runs_per_instance * iterations_per_run
for start in range(0, df.shape[0], problem_size):
    print(" / problem type start: ", start)
    df_by_problem_type = df.iloc[start:start + problem_size]
    # divide in two, first part was done using roulette-wheel, second part was done using random selector
    df_roulette_wheel = df_by_problem_type.iloc[0:18000]
    df_random = df_by_problem_type.iloc[18000:]
    # from each problem type take three instances - two for validation and one for testing
    # training sets:
    df_roulette_wheel_train = df_roulette_wheel.iloc[0:2000*6]
    df_random_train = df_random.iloc[0:2000*6]
    # validation sets:
    df_roulette_wheel_val = df_roulette_wheel.iloc[2000*6:2000*8]
    df_random_val = df_random.iloc[2000*6:2000*8]
    # testing sets:
    df_roulette_wheel_test = df_roulette_wheel.iloc[2000*8:]
    df_random_test = df_random.iloc[2000*8:]

    print("train length of both operator selectors: ", df_roulette_wheel_train.shape[0], df_random_train.shape[0])
    print("validation length of both operator selectors: ", df_roulette_wheel_val.shape[0], df_random_val.shape[0])
    print("test length of both operator selectors: ", df_roulette_wheel_test.shape[0], df_random_test.shape[0])

    training_subset = pd.concat([df_roulette_wheel_train, df_random_train])
    training_set.append(training_subset)

    validation_subset = pd.concat([df_roulette_wheel_val, df_random_val])
    validation_set.append(validation_subset)

    test_subset = pd.concat([df_roulette_wheel_test, df_random_test])
    training_set.append(test_subset) #todo: i changed to the training subset so it has more data to train with

    # process_data(df_subset)

 / problem type start:  0
train length of both operator selectors:  12000 12000
validation length of both operator selectors:  4000 4000
test length of both operator selectors:  2000 2000
 / problem type start:  36000
train length of both operator selectors:  12000 12000
validation length of both operator selectors:  4000 4000
test length of both operator selectors:  2000 2000
 / problem type start:  72000
train length of both operator selectors:  12000 12000
validation length of both operator selectors:  4000 4000
test length of both operator selectors:  2000 2000
 / problem type start:  108000
train length of both operator selectors:  12000 12000
validation length of both operator selectors:  4000 4000
test length of both operator selectors:  2000 2000
 / problem type start:  144000
train length of both operator selectors:  12000 12000
validation length of both operator selectors:  4000 4000
test length of both operator selectors:  2000 2000
 / problem type start:  180000
train lengt

In [74]:
training_set   = pd.concat(training_set, ignore_index=True)
validation_set = pd.concat(validation_set,   ignore_index=True)
#test_set       = pd.concat(test_set,  ignore_index=True)

In [75]:
print(training_set.shape)
print(validation_set.shape)
# print(test_set.shape)

(168000, 27)
(48000, 27)


In [82]:
training_set.to_csv('training_set_2.csv', header=True, index=False)
validation_set.to_csv('validation_set_2.csv', header=True, index=False)
#test_set.to_csv('test_set.csv', header=True, index=False)

In [50]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(training_set[["short_long_improvement"]])
training_set["y_scaled"] = scaler.transform(training_set[["short_long_improvement"]])

In [48]:
# Group data that belongs to the same state
same_state_cols = ["instance_type","tw_spread", "chosen_remove_operator", "chosen_insert_operator"]
grp_same_state = training_set.groupby(same_state_cols)
print("number of groups= ", grp_same_state.ngroups)
# for a total of 144,000 entries in training_set, 105442 groups were created, which means, there are not many entries that share the same state

number of groups=  90


In [51]:
noise_var  = grp_same_state["y_scaled"].var().mean()      # σ²_noise
print(noise_var)

0.8446677456447272


In [52]:
training_set["y_scaled"].var()

np.float64(1.0000059524163836)

## Prepare scalers for the data

In [53]:
import joblib
from sklearn.compose      import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler

In [54]:
train_df = pd.read_csv("training_set.csv")

In [64]:
train_df.head()


,iterations,instance_type,tw_spread,operator_selection_mechanism,number_of_vertices_to_remove,delta_last_improv,acceptance_ratio,i_last_improv,prev_remove_operator,prev_insert_operator,...,success_i_op_1,success_i_op_2,success_i_op_3,delta_cost,new_cost,chosen_remove_operator,chosen_insert_operator,current_cost,rel_delta_last_improv,short_long_improvement
0,1,0,1,1,15,0.000000,0.0,0,NaN,NaN,...,1.0,1.0,1.0,-718.948672,9548.083839,1,2,10267.032510,0.000000,-0.054251
1,2,0,1,1,5,-718.948672,1.0,0,1.0,2.0,...,1.0,1.0,1.0,-91.810910,9456.272928,1,3,9548.083839,-0.075298,-0.012359
2,3,0,1,1,5,-91.810910,1.0,0,1.0,3.0,...,1.0,1.0,1.0,-160.354919,9295.918009,4,1,9456.272928,-0.009709,-0.017553
3,4,0,1,1,5,-160.354919,1.0,0,4.0,1.0,...,1.0,1.0,1.0,0.000000,9295.918009,5,2,9295.918009,-0.017250,-0.005781
4,5,0,1,1,5,-160.354919,1.0,1,5.0,2.0,...,1.0,1.0,1.0,0.000000,9295.918009,3,1,9295.918009,-0.017250,-0.005781


In [66]:
# Prepare scaler for input features
num_pos    = ["iterations", "acceptance_ratio", "number_of_vertices_to_remove", "i_last_improv", "route_imbalance", "capacity_utilization", 'success_r_op_1', 'success_r_op_2',                     'success_r_op_3', 'success_r_op_4', 'success_r_op_5', 'success_i_op_1', 'success_i_op_2','success_i_op_3']      # always positive

num_signed = ["rel_delta_last_improv"]           # can be positive or negative

cat_cols   = ["instance_type", "tw_spread", "operator_selection_mechanism",
              "prev_remove_operator", "prev_insert_operator"]             # categorical columns

# ❶  feature transformer (dense output for easy → torch)
prep = ColumnTransformer(
        transformers=[
            ("pos",    MinMaxScaler(),               num_pos),
            ("signed", MinMaxScaler((-1, 1)),        num_signed),
            ("cat",    OneHotEncoder(sparse_output=False,
                                     handle_unknown="ignore"), cat_cols),
        ])

prep.fit(train_df[num_pos + num_signed + cat_cols])
joblib.dump(prep, "../artifacts/feature_prep.joblib")

['artifacts/feature_prep.joblib']

In [68]:
# prepare scaler for target feature
y_scaler = StandardScaler().fit(
              train_df[["short_long_improvement"]])

joblib.dump(y_scaler, "../artifacts/y_scaler.joblib")

['artifacts/y_scaler.joblib']

## OLd: Now split into batches, 6-2-1

In [67]:
# First I will train the NN using only C1
c1_roulette_wheel_training = df.iloc[0:2000*6]
c1_random_training = df.iloc[2000*9:2000*9+2000*6]

In [68]:
c1_roulette_wheel_validation = df.iloc[2000*6:2000*8]
c1_random_validation = df.iloc[2000*9+2000*6:2000*9+2000*8]

In [69]:
c1_roulette_wheel_test = df.iloc[2000*8:2000*9]
c1_random_test = df.iloc[2000*9+2000*8:2000*9+2000*9]

In [37]:
len(c1_roulette_wheel_test)

2000

In [41]:
c1_random_test.head()

,iterations,instance_type,tw_spread,operator_selection_mechanism,number_of_vertices_to_remove,delta_last_improv,acceptance_ratio,i_last_improv,prev_remove_operator,prev_insert_operator,...,success_r_op_4,success_r_op_5,success_i_op_1,success_i_op_2,success_i_op_3,delta_cost,new_cost,chosen_remove_operator,chosen_insert_operator,short_long_improvement
34000,1,0,1,0,15,0.000000,0.0,0,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,-92.845866,9414.022224,3,2,-359.520402
34001,2,0,1,0,5,-92.845866,1.0,0,3.0,2.0,...,1.0,1.0,1.0,1.0,1.0,0.000000,9414.022224,5,2,-294.528296
34002,3,0,1,0,5,-92.845866,1.0,1,5.0,2.0,...,1.0,1.0,1.0,1.0,1.0,-0.754637,9413.267587,5,1,-295.056541
34003,4,0,1,0,5,-0.754637,1.0,0,5.0,1.0,...,1.0,1.0,1.0,1.0,1.0,-3.333590,9409.933997,1,1,-296.861809
34004,5,0,1,0,5,-3.333590,1.0,0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,-340.195001,9069.738996,5,2,-532.664796


In [70]:
c1_training = pd.concat([c1_roulette_wheel_training, c1_random_training], ignore_index=True)
c1_validation = pd.concat([c1_roulette_wheel_validation, c1_random_validation], ignore_index=True)
c1_test = pd.concat([c1_roulette_wheel_test, c1_random_test], ignore_index=True)

In [71]:
c1_training.to_csv('c1_training.csv', header=True, index=False)
c1_validation.to_csv('c1_validation.csv', header=True, index=False)
c1_test.to_csv('c1_test.csv', header=True, index=False)

,iterations,instance_type,tw_spread,operator_selection_mechanism,number_of_vertices_to_remove,delta_last_improv,acceptance_ratio,i_last_improv,prev_remove_operator,prev_insert_operator,...,success_r_op_4,success_r_op_5,success_i_op_1,success_i_op_2,success_i_op_3,delta_cost,new_cost,chosen_remove_operator,chosen_insert_operator,short_long_improvement
0,1,0,1,1,15,0.000000,0.0,0,NaN,NaN,...,1.000000,1.00000,1.000000,1.000000,1.000000,-718.948672,9548.083839,1,2,-557.001712
1,2,0,1,1,5,-718.948672,1.0,0,1.0,2.0,...,1.000000,1.00000,1.000000,1.000000,1.000000,-91.810910,9456.272928,1,3,-118.005279
2,3,0,1,1,5,-91.810910,1.0,0,1.0,3.0,...,1.000000,1.00000,1.000000,1.000000,1.000000,-160.354919,9295.918009,4,1,-165.986085
3,4,0,1,1,5,-160.354919,1.0,0,4.0,1.0,...,1.000000,1.00000,1.000000,1.000000,1.000000,0.000000,9295.918009,5,2,-53.737642
4,5,0,1,1,5,-160.354919,1.0,1,5.0,2.0,...,1.000000,1.00000,1.000000,1.000000,1.000000,0.000000,9295.918009,3,1,-53.737642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,1996,0,1,0,15,-334.782193,1.0,1,5.0,3.0,...,3.966727,1.48684,2.043768,3.097418,2.689378,-723.808981,12289.819320,1,2,-1011.110096
23996,1997,0,1,0,15,-723.808981,1.0,0,1.0,2.0,...,3.966727,1.48684,2.043768,3.097418,2.689378,-1681.479365,10608.339955,4,2,-1347.567058
23997,1998,0,1,0,15,-1681.479365,1.0,0,4.0,2.0,...,3.966727,1.48684,2.043768,3.097418,2.689378,242.856720,10851.196676,2,3,-0.531799
23998,1999,0,1,0,15,-1681.479365,1.0,1,2.0,3.0,...,3.966727,1.48684,2.043768,3.097418,2.689378,-198.515670,10652.681006,2,3,-309.492472
